In [18]:
import numpy as np
import pandas as pd

In [19]:
training_data = pd.read_csv('df_train.csv')
test_data = pd.read_csv('df_test.csv')

In [20]:
training_data.head()

,Unnamed: 0,url,Label,label_code,use_of_ip,count.,∼ Precence,@ Precence,- Precence,count_embed_domian,sus_url,short_url,HTTPS in Domain,url_length,tld,tld_length
0,0,princess-diana.com/,good,1,0,1,0,0,1,0,0,0,0,19,NaN,-1
1,1,filmonlineforum.net/index.php?topic=5012.0,good,1,0,3,0,0,0,0,0,0,0,42,NaN,-1
2,2,mojavelofts.com/del/,good,1,0,1,0,0,0,0,0,0,0,20,NaN,-1
3,3,http://www.brigitte-koischwitz.de/index.php?op...,bad,0,0,3,0,0,1,0,0,0,0,91,de,2
4,4,oldies.com/product-view/2199GD.html,good,1,0,2,0,0,1,0,0,0,0,35,NaN,-1


In [21]:
class NN(object):
    def __init__(self):
        self.input = 10
        self.output = 1
        self.hidden_units = 11

        np.random.seed(1)
        self.w1 = np.random.randn(self.input, self.hidden_units) * 0.01
        self.w2 = np.random.randn(self.hidden_units, self.output) * 0.01

    def save_model(self, w1_file='w1.npy', w2_file='w2.npy'):
        np.save(w1_file, self.w1)
        np.save(w2_file, self.w2)
        print("Model weights saved successfully.")

    def load_model(self, w1_file='w1.npy', w2_file='w2.npy'):
        self.w1 = np.load(w1_file)
        self.w2 = np.load(w2_file)
        print("Model weights loaded successfully.")

    def _forward_propagation(self, X):
        self.z2 = np.dot(X, self.w1)
        self.a2 = self._sigmoid(self.z2)
        self.z3 = np.dot(self.a2, self.w2)
        self.a3 = self._sigmoid(self.z3)
        return self.a3

    def _sigmoid(self, z):
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))

    def _loss(self, predict, y):
        m = y.shape[0]
        logprobs = y * np.log(predict) + (1 - y) * np.log(1 - predict)
        loss = -np.sum(logprobs) / m
        return loss

    def _backward_propagation(self, X, y):
        predict = self.a3
        m = X.shape[0]


        delta3 = predict - y
        self.dw2 = np.dot(self.a2.T, delta3) / m
        db2 = np.sum(delta3, axis=0) / m


        delta2 = np.dot(delta3, self.w2.T) * self._sigmoid_prime(self.z2)
        self.dw1 = np.dot(X.T, delta2) / m
        db1 = np.sum(delta2, axis=0) / m


        self.w1 -= self.learning_rate * self.dw1
        self.w2 -= self.learning_rate * self.dw2

    def _sigmoid_prime(self, z):
        return self._sigmoid(z) * (1 - self._sigmoid(z))

    def train(self, X, y, iterations=33, learning_rate=1.2):
        self.learning_rate = learning_rate
        y = y.values.reshape(-1, 1)

        for i in range(iterations):
            y_hat = self._forward_propagation(X)
            loss = self._loss(y_hat, y)
            self._backward_propagation(X, y)

            if i % 10 == 0:
                print(f"Iteration {i}, Loss: {loss}")

    def predict(self, X):
        y_hat = self._forward_propagation(X)
        y_hat = (y_hat >= 0.5).astype(int)
        return y_hat

    def score(self, y_pred, y_true):
        return (np.mean(y_pred == y_true.values.reshape(-1, 1))) * 100

if __name__=='__main__':
    train_X = training_data[['use_of_ip', 'count.', '@ Precence', '- Precence', '∼ Precence', 'count_embed_domian', 'sus_url', 'short_url', 'HTTPS in Domain', 'url_length']]
    test_X =  test_data[['use_of_ip', 'count.', '@ Precence', '- Precence', '∼ Precence', 'count_embed_domian', 'sus_url', 'short_url', 'HTTPS in Domain', 'url_length']]
    train_y = training_data['label_code']
    test_y = test_data['label_code']

    clr = NN()
    clr.train(train_X, train_y, iterations=100)
    pre_y = clr.predict(test_X)
    score = clr.score(pre_y, test_y)
    print("Predictions:", pre_y.flatten())
    print("Actual:", test_y.values)
    print("Accuracy Score:", score)
    clr.save_model()

Iteration 0, Loss: 0.6953908281156956
Iteration 10, Loss: 0.6227227817996747
Iteration 20, Loss: 0.6221202826047213
Iteration 30, Loss: 0.622672907797841
Iteration 40, Loss: 0.6244371487729662
Iteration 50, Loss: 0.6236193222661437
Iteration 60, Loss: 0.6231138833338137
Iteration 70, Loss: 0.6237807236222739
Iteration 80, Loss: 0.62549737125725
Iteration 90, Loss: 0.6235095553713949
Predictions: [1 1 1 ... 1 1 1]
Actual: [0 1 0 ... 1 0 1]
Accuracy Score: 68.36931034913601
Model weights saved successfully.
